<a href="https://colab.research.google.com/github/arifsoul/Keras-MultiClass-Image-Classification/blob/master/Train_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Classification SVM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Split folder

In [ ]:
#split folder train and val
#!rm -rf /content/drive/MyDrive/Datasets/Classifier/split
!pip install split-folders
import splitfolders 
data_dir = '/content/drive/MyDrive/Datasets/Classifier/zMerge'
splitfolders.ratio(data_dir, output="/content/drive/MyDrive/Datasets/Classifier/split", ratio=(0.8, 0.2), group_prefix=None)

Copying files: 3317 files [10:10,  5.43 files/s]


##Training

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
train_dir = "/content/drive/MyDrive/Datasets/Classifier/split/train"
test_df = "/content/drive/MyDrive/Datasets/Classifier/split/val" 
os.listdir(train_dir)

['casiterite_hulu_biat',
 'casiterite_air_landik',
 'casiterite_merah',
 'anatase',
 'casiterite_laut_modong',
 'biotite',
 'casiterite',
 'barite',
 'casiterite_balembang',
 'calcopyrite',
 'galena',
 'hematite',
 'casiterite_nigeria',
 'casiterite_singkep',
 'ilmenite',
 'casiterite_pulau_lalang',
 'kalsit',
 'hematite2',
 'garnet',
 'corundum',
 'lempung_pasir',
 'oksida_besi',
 'muscovite',
 'magnetite',
 'leucoxene',
 'limonite',
 'monazite',
 'phyrite',
 'psilomelane',
 'metal',
 'tourmaline',
 'siderite',
 'rutile',
 'topaz',
 'tera',
 'slag',
 'wolframite',
 'quartz',
 'spinel',
 'xenotime',
 'zircon']

In [ ]:
train_datagen = ImageDataGenerator(rescale=(1/255.),shear_range = 0.2,zoom_range=0.2,
                                   horizontal_flip=True)
training_set = train_datagen.flow_from_directory(directory = train_dir,target_size=(64,64),
                                                batch_size=5,
                                                class_mode = "categorical")
test_datagen = ImageDataGenerator(rescale=(1/255.))
test_set = test_datagen.flow_from_directory(directory = test_df,target_size=(64,64),
                                                batch_size=5,
                                                class_mode = "categorical")

Found 2635 images belonging to 41 classes.
Found 682 images belonging to 41 classes.


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten
from tensorflow.keras.regularizers import l2

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3, strides = 2,input_shape=(64,64,3)))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Conv2D(filters = 32, padding = "same",activation = "relu",kernel_size=3))
model.add(MaxPool2D(pool_size=(2,2),strides = 2))

model.add(Flatten())
model.add(Dense(128,activation="relu"))

#Output layer
model.add(Dense(41,kernel_regularizer='l2',activation= "softmax"))

# Compiling the CNN
model.compile(optimizer="adam",loss="squared_hinge", metrics = ['accuracy'])

In [ ]:
!pip install livelossplot
from livelossplot import PlotLossesKeras

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.93):
      print("\nAkurasi telah mencapai >93%!")
      self.model.stop_training = True
callbacks = myCallback()

In [ ]:
#Training
# history = model.fit(x = training_set,steps_per_epoch=100, epochs=500,validation_data = test_set)

epochs=500
history = model.fit(
  training_set,
  validation_data=test_set,
  epochs=epochs, 
  callbacks=[PlotLossesKeras(), callbacks]
)

Epoch 1/500
100/100 [==============================] - 5s 55ms/step - loss: 1.0241 - accuracy: 0.4680 - val_loss: 1.0223 - val_accuracy: 0.5000
Epoch 2/500
100/100 [==============================] - 6s 56ms/step - loss: 1.0185 - accuracy: 0.5480 - val_loss: 1.0217 - val_accuracy: 0.5015
Epoch 3/500
100/100 [==============================] - 5s 55ms/step - loss: 1.0192 - accuracy: 0.5360 - val_loss: 1.0208 - val_accuracy: 0.5132
Epoch 4/500
100/100 [==============================] - 6s 56ms/step - loss: 1.0188 - accuracy: 0.5440 - val_loss: 1.0213 - val_accuracy: 0.5029
Epoch 5/500
100/100 [==============================] - 5s 54ms/step - loss: 1.0197 - accuracy: 0.5480 - val_loss: 1.0222 - val_accuracy: 0.5103
Epoch 6/500
100/100 [==============================] - 6s 56ms/step - loss: 1.0186 - accuracy: 0.5260 - val_loss: 1.0203 - val_accuracy: 0.5220
Epoch 7/500
100/100 [==============================] - 6s 58ms/step - loss: 1.0167 - accuracy: 0.5760 - val_loss: 1.0193 - val_accuracy:

In [ ]:
model.save("/content/drive/MyDrive/Datasets/Classifier/saved_model-SVM/svm-mineral-rocks-v2.h5")